In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # Colab 환경설정

!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

In [ ]:
!nvidia-smi

In [ ]:
!kill -9 1

In [3]:
# !pip install matplotlib

In [18]:
# # github에서 Kobert 파일 로드 및 Kobert 모델 불러오기

# #kobert_tokenizer 폴더 다운받는 코드
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

# #https://github.com/SKTBrain/KoBERT 의 파일들을 다운받는 코드
# !pip install 'git+https://git@github.com/SKTBrain/KoBERT.git@master'

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import gluonnlp as nlp

model_name = "skt/kobert-base-v1"
tokenizer = KoBERTTokenizer.from_pretrained(model_name, use_fast=False)
bertmodel = BertModel.from_pretrained(model_name)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [19]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [20]:
# 필요한 라이브러리 불러오기
# import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import re

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용 시
device = torch.device("cuda:0")

In [21]:
# 입력 데이터셋을 토큰화하기

# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding padding 등을 해주는 코드이다.
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [23]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [31]:
# from sklearn.model_selection import train_test_split

# target = df['Emotion']
# # train_test_split
# df_train, df_val = train_test_split(df, test_size=0.2, shuffle=True, stratify=target, random_state=34)
# df_train.reset_index(drop=True, inplace=True)
# df_val.reset_index(drop=True, inplace=True)

In [32]:
# df_train.to_csv("./dataset/new_train_all.csv", encoding="utf-8-sig", index=False)
# df_val.to_csv("./dataset/new_validation_all.csv", encoding="utf-8-sig", index=False)

In [33]:
df_train = pd.read_csv("./dataset/new_train_all.csv", encoding='utf-8')
df_val = pd.read_csv("./dataset/new_validation_all.csv", encoding='utf-8')

In [34]:
df_train

,발화문,Emotion
0,3주 정도 된 것 같아,neutral
1,걱정할 것 같아서 얘기를 못 하겠어,sadness
2,지금은 많이 좋아졌어 하지만 또 천둥 칠까봐 무섭긴 해,fear
3,너무 마음에 들어,happiness
4,비싼 향수를 당첨 선물로 받았다고 주위 사람들한테 자랑했어 다들 부러워 하더라고,neutral
...,...,...
31896,조언해줘서 고마워,happiness
31897,맞아 밖에 나가서 햇빛 좀 받으면서 활동해야 한다고,neutral
31898,친구랑 약속 있어 나왔는데 친구가 아직도 오질 않고 있어,angry
31899,좋은 곳으로 갔을 거야 내 말 들어줘서 고마워,happiness


In [35]:
df_val

,발화문,Emotion
0,나 결국 남자친구랑 헤어졌어,neutral
1,네 그래야될 거 같아요 손해가 너무 많아가지고,sadness
2,집에서 나올 때 우산 챙겨오면 돼 걱정하지 마 건강은 괜찮지,neutral
3,어 그거 괜찮은 생각이다 좋은 조언해줘서 고마워,happiness
4,그러게 안그래도 더 그런 것 같다 장마 때문에 아 진짜 어떡하지,angry
...,...,...
7971,응 지금은 조금 진정됐어 근데 어젯밤에는 너무너무 깜짝놀래서 진짜 놀랬지 뭐야,neutral
7972,퇴근한 지 얼마나 됐다고 회사에서 또 전화가 왔어,neutral
7973,내가 재빠르게 달려가서 해피를 안고 와서 아무일도 없었어 다행이었어,happiness
7974,지금 30분 넘었어,neutral


In [36]:
df_train = df_train.replace({"Emotion":{"neutral": 0, "happiness": 1, "sadness": 2, "angry": 3, "disgust": 4, "fear": 5, "surprise": 6}})
df_val = df_val.replace({"Emotion":{"neutral": 0, "happiness": 1, "sadness": 2, "angry": 3, "disgust": 4, "fear": 5, "surprise": 6}})

In [37]:
df_train

,발화문,Emotion
0,3주 정도 된 것 같아,0
1,걱정할 것 같아서 얘기를 못 하겠어,2
2,지금은 많이 좋아졌어 하지만 또 천둥 칠까봐 무섭긴 해,5
3,너무 마음에 들어,1
4,비싼 향수를 당첨 선물로 받았다고 주위 사람들한테 자랑했어 다들 부러워 하더라고,0
...,...,...
31896,조언해줘서 고마워,1
31897,맞아 밖에 나가서 햇빛 좀 받으면서 활동해야 한다고,0
31898,친구랑 약속 있어 나왔는데 친구가 아직도 오질 않고 있어,3
31899,좋은 곳으로 갔을 거야 내 말 들어줘서 고마워,1


In [38]:
df_val

,발화문,Emotion
0,나 결국 남자친구랑 헤어졌어,0
1,네 그래야될 거 같아요 손해가 너무 많아가지고,2
2,집에서 나올 때 우산 챙겨오면 돼 걱정하지 마 건강은 괜찮지,0
3,어 그거 괜찮은 생각이다 좋은 조언해줘서 고마워,1
4,그러게 안그래도 더 그런 것 같다 장마 때문에 아 진짜 어떡하지,3
...,...,...
7971,응 지금은 조금 진정됐어 근데 어젯밤에는 너무너무 깜짝놀래서 진짜 놀랬지 뭐야,0
7972,퇴근한 지 얼마나 됐다고 회사에서 또 전화가 왔어,0
7973,내가 재빠르게 달려가서 해피를 안고 와서 아무일도 없었어 다행이었어,1
7974,지금 30분 넘었어,0


In [39]:
df_val['Emotion'].value_counts(normalize=True)

0    0.363591
2    0.241976
3    0.150702
1    0.125502
4    0.052533
5    0.051153
6    0.014544
Name: Emotion, dtype: float64

In [40]:
df_val['Emotion'].unique()

array([0, 2, 1, 3, 5, 4, 6])

In [41]:
dataset_train = []
for ques, label in zip(df_train['발화문'], df_train['Emotion']):
    data = []
    data.append(ques)
    data.append(label)
    
    dataset_train.append(data)
    
dataset_val = []
for ques, label in zip(df_val['발화문'], df_val['Emotion']):
    data = []
    data.append(ques)
    data.append(label)
    
    dataset_val.append(data)

In [ ]:
dataset_train

In [ ]:
dataset_val

In [44]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_validation = BERTDataset(dataset_val,0, 1, tok, vocab,  max_len, True, False)

In [45]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
validation_dataloader = torch.utils.data.DataLoader(data_validation, batch_size=batch_size, num_workers=0)

In [46]:
# KoBERT 모델 구현하기

In [47]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [48]:
df_train['Emotion'].value_counts()

0    11597
2     7720
3     4806
1     4003
4     1675
5     1633
6      467
Name: Emotion, dtype: int64

In [49]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [50]:
# 오차행렬, 정확도, 정밀도, 재현율 f1 score을 한번에 출력하기 위한 함수 정의

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_eval(y_true, y_pred):
#     confusion = confusion_matrix(y_test, pred)
#     answer = torch.cat(y_test).detach().cpu().tolist()
#     temp = torch.cat(pred).detach().cpu().tolist()
#     prediction = []
#     for i in range(len(temp)):
#         prediction.append(temp[i].index(max(temp[i])))
    answer = y_true
    prediction = y_pred
    accuracy = accuracy_score(answer, prediction)
    precision = precision_score(answer, prediction, average= "weighted")
    recall = recall_score(answer, prediction, average= "weighted")
    f1 = f1_score(answer, prediction, average= "weighted")
#     print('Confusion Matrix')
#     print(confusion)
    print('정확도:{}, 정밀도:{}, 재현율:{} f1 score:{}'.format(accuracy, precision, recall, f1))
    text = '정확도:{}, 정밀도:{}, 재현율:{} f1 score:{}'.format(accuracy, precision, recall, f1)
    return text

In [51]:
print(len(dataset_train))

31901


In [52]:
# 학습 모델 저장할 폴더 생성
import os
PATH = f'./Model/batch size={batch_size}, learning rate={learning_rate}/'
os.mkdir(PATH)

In [53]:
# train

In [54]:
# train

x_true = []
x_pred = []
y_true = []
y_pred = []
train_history=[]
validation_history=[]
loss_history=[]
validation_loss_history=[]
for e in range(num_epochs):
    # epoch별 모델 저장할 폴더 생성
    MODEL_PATH = PATH + f'epoch{e+1}/'
    os.mkdir(MODEL_PATH)
    
#     train_acc = 0.0
#     val_acc = 0.0
    train_acc = 0
    val_acc = 0
    model.train()
    print(f"Epoch {e+1} Train")
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        x_true.extend(label.cpu().detach().numpy().tolist())
        x_pred.extend(torch.argmax(out, dim=1).cpu().detach().tolist())
        # print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
        train_acc = accuracy_score(x_true, x_pred)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc))
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     train_history.append(train_acc / (batch_id+1))
    train_history.append(train_acc)
    loss_history.append(loss.data.cpu().numpy())
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc))
    print()
#     train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    with torch.no_grad():
    #         test_loss = 0
        print(f"Epoch {e+1} Validation")
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(validation_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            y_true.extend(label.cpu().detach().numpy().tolist())    # 정답 y_true 리스트에 cpu로 저장
            out = model(token_ids, valid_length, segment_ids)
#             print(f"y_true : {label}")
#             print(f"y_true : {out}")
            y_pred.extend(torch.argmax(out, dim=1).cpu().detach().tolist())    # 예측값 클래스별 가중치 y_pred 리스트에 cpu로 저장

            loss = loss_fn(out, label)
#             val_acc += calc_accuracy(out, label)
            val_acc = accuracy_score(y_true, y_pred)
            if batch_id % log_interval == 0:
#                 print("epoch {} batch id {} loss {} validation acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), val_acc / (batch_id+1)))
                print("epoch {} batch id {} loss {} validation acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), val_acc))
#         validation_history.append(val_acc / (batch_id+1))
        validation_history.append(val_acc)
        validation_loss_history.append(loss.data.cpu().numpy())

    #         val_loss += loss_fn(out, label).data.cpu().numpy()

#         print("epoch {} validation acc {} ".format(e+1, val_acc / (batch_id+1)))
        print("epoch {} validation acc {} ".format(e+1, val_acc))
    #         validation_history.append(val_acc / (batch_id+1))

        get_clf_eval(y_true, y_pred)

        # 학습 모델 저장
        torch.save(model, MODEL_PATH + 'Model.pt')  # 전체 모델 저장
#         torch.save(model.state_dict(), MODEL_PATH + 'Model_state_dict.pt')  # 모델 객체의 state_dict 저장

        
    

Epoch 1 Train


  0%|          | 0/499 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9512392282485962 train acc 0.15625
epoch 1 batch id 201 loss 1.21956205368042 train acc 0.40220771144278605
epoch 1 batch id 401 loss 0.74610835313797 train acc 0.5559149002493765
epoch 1 train acc 0.5960001253879189

Epoch 1 Validation


  0%|          | 0/125 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6275519132614136 validation acc 0.796875
epoch 1 validation acc 0.7917502507522568 
정확도:0.7917502507522568, 정밀도:0.79354697515032, 재현율:0.7917502507522568 f1 score:0.7895298290951073
Epoch 2 Train


...

epoch 9 batch id 1 loss 0.6355103254318237 validation acc 0.8386930110220441
epoch 9 validation acc 0.841106653293213 
정확도:0.841106653293213, 정밀도:0.8411075797974195, 재현율:0.841106653293213 f1 score:0.8408644217958614
Epoch 10 Train


  0%|          | 0/499 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0023728571832180023 train acc 0.8894150912516149
epoch 10 batch id 201 loss 0.013896630145609379 train acc 0.8937037666723339
epoch 10 batch id 401 loss 0.012134980410337448 train acc 0.897692575765811
epoch 10 train acc 0.8995548728879973

Epoch 10 Validation


  0%|          | 0/125 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.6052495241165161 validation acc 0.8411090079055784
epoch 10 validation acc 0.8430792377131394 
정확도:0.8430792377131394, 정밀도:0.8430942202971989, 재현율:0.8430792377131394 f1 score:0.8428533669597097


In [ ]:
# 직접 만든 새로운 문장으로 테스트

In [55]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("중립이")
            elif np.argmax(logits) == 1:
                test_eval.append("행복이")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 5:
                test_eval.append("공포가")
            elif np.argmax(logits) == 6:
                test_eval.append("놀람이")



        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [56]:
# 질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 오늘 마지막 시험을 쳤다.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 피곤하지만 이제 다 끝났다 생각하니깐 기분이 좋다.
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 동기들이랑 시험 끝나고 닭한마리도 먹고, 카페에서 팀플도 했다.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 제발 오류나지 말고 잘 되었으면 좋겠다.
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 엔터 쓰면 안 된다.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 구두점을 잘 찍으면 좋다!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 버그가 발견돼서 슬프다.
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 새로 업데이트가 되었다니 너무 기쁘네요!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 내 나무가 사라졌었어요ㅠ
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 근데 이유를 모르겠어요,,,드디어 주말이네용.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 행복한 주말을 보내야겠어요!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 내 나무가 사라졌었어요.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 그래서 슬펐어요.
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 슬픈 감정 포인트를 받고 싶었는데 중립으로 떠서 또 우울했어요.
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 디비를 삭제하고 다시 써요.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 드디어 주말이네용.
>> 입력하신 내용에서 중립이 느껴집니다.


하고

KeyboardInterrupt: Interrupted by user